## Presets

In [1]:
import os, sys
PWD = os.getenv('PWD')
os.chdir(PWD)
sys.path.insert(0, os.getenv('PWD'))
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "edusite/settings.py")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
import django
django.setup()

---

In [2]:
from expenses.models import BankAccount

from datetime import timedelta
from django.utils import timezone

In [3]:
import random
from collections import Counter

In [4]:
# for raw queries to DB
from django.db import connection

---

In [5]:
def dictfetchall(cursor):
    "Return all rows from a cursor as a dict"
    columns = [col[0] for col in cursor.description]
    return [
        dict(zip(columns, row))
        for row in cursor.fetchall()
    ]

In [6]:
with connection.cursor() as cursor:
        cursor.execute("EXPLAIN ANALYZE SELECT * FROM expenses_bankaccount")
        for i in dictfetchall(cursor):
            print(i)

{'QUERY PLAN': 'Seq Scan on expenses_bankaccount  (cost=0.00..1.03 rows=3 width=29) (actual time=0.021..0.026 rows=3 loops=1)'}
{'QUERY PLAN': 'Planning Time: 0.740 ms'}
{'QUERY PLAN': 'Execution Time: 0.118 ms'}


## Setting up

In [7]:
accounts = BankAccount.objects.all()
accounts

<QuerySet [<BankAccount: Bank account #3
            Mandalorian 
            (2022-12-06) 
            with $988760>, <BankAccount: Bank account #1
            admin acc 
            (2022-12-06) 
            with $4012510>, <BankAccount: Bank account #2
            Ivan 
            (2022-12-06) 
            with $2072160>]>

In [8]:
def send_money_direct(sender_account, money, recipient_account, pub_date):
    new_ticket = Ticket()
    new_ticket.account = sender_account
    new_ticket.ticket_text = 'sending money to acc #' + str(recipient_account.pk)
    new_ticket.pub_date = pub_date
    new_ticket.money = money
    new_ticket.save()
    recipient_account.money += money
    recipient_account.save()

In [9]:
def create_ticket_direct(account, money, description, pub_date):
    new_ticket = Ticket()
    new_ticket.account = account
    new_ticket.ticket_text = description
    new_ticket.pub_date = pub_date
    new_ticket.money = money
    new_ticket.save()

In [10]:
# create_ticket_direct(acc, 10, "soup", get_time_delta(0))
# send_money_direct(acc1, 20, acc0, get_time_delta(2))

In [11]:
def get_time_delta(deltahours):
    return timezone.now() - timedelta(hours=deltahours)

In [12]:
# get_time_delta(0), get_time_delta(1)

## Generating pipeline

In [13]:
accounts = BankAccount.objects.all()
accounts

<QuerySet [<BankAccount: Bank account #3
            Mandalorian 
            (2022-12-06) 
            with $988760>, <BankAccount: Bank account #1
            admin acc 
            (2022-12-06) 
            with $4012510>, <BankAccount: Bank account #2
            Ivan 
            (2022-12-06) 
            with $2072160>]>

In [14]:
# for acc in accounts:
#     acc.money *= 10
#     acc.save()

In [18]:
%%time 

k = 100000
deltahours = k # hours ago
action_list = ["create_ticket", "send_money_direct", "send_money_direct"] # weighted-like actions 1:2
money_list = [1,1,2,2,5,5,10] # weighted-like money 2:2:2:1
tickets_descriptions = ["pizza", "sushi", "soup", "salad"]

act_log = []
res_log = []

for acc in random.choices(accounts, weights=[2,5,1], k=k):
    
    act = random.choice(action_list)
    mon = random.choice(money_list)
    
    if act == "send_money_direct":
        # choose a recipient account
        other_accs = list(filter(lambda x: x != acc, accounts))
        to_acc = random.choice(other_accs)
        # log action
        log_text = f"sending from <{acc.account_text}> to <{to_acc.account_text}> ${mon}"
        act_log.append(act)
        res_log.append(log_text)
        
        # send money
        send_money_direct(acc, mon, to_acc, get_time_delta(deltahours))
        
    elif act == "create_ticket":
        # choose a description of the ticket
        text = random.choice(tickets_descriptions)
        # log action
        log_text = f"creating ticket for <{acc.account_text}>: {text} ${mon}"
        act_log.append(act)
        res_log.append(log_text)
        
        # create ticket
        create_ticket_direct(acc, mon, text, get_time_delta(deltahours))
    
    # decrease time delta
    deltahours -= random.choice([0,0,1])

CPU times: user 6min 16s, sys: 25.1 s, total: 6min 41s
Wall time: 11min 2s


## Result

In [19]:
Counter(act_log)

Counter({'send_money_direct': 66756, 'create_ticket': 33244})

In [20]:
Counter(res_log)

Counter({'sending from <admin acc> to <Ivan> $5': 5905,
         'sending from <admin acc> to <Ivan> $10': 2980,
         'sending from <admin acc> to <Mandalorian> $2': 5900,
         'sending from <admin acc> to <Ivan> $1': 5923,
         'creating ticket for <Ivan>: pizza $1': 312,
         'sending from <Mandalorian> to <Ivan> $1': 2336,
         'creating ticket for <admin acc>: sushi $10': 745,
         'sending from <Mandalorian> to <admin acc> $10': 1262,
         'creating ticket for <admin acc>: soup $2': 1392,
         'creating ticket for <Mandalorian>: salad $1': 591,
         'sending from <Mandalorian> to <admin acc> $1': 2427,
         'sending from <Mandalorian> to <admin acc> $5': 2420,
         'sending from <admin acc> to <Mandalorian> $5': 6067,
         'creating ticket for <admin acc>: pizza $5': 1395,
         'creating ticket for <admin acc>: pizza $1': 1521,
         'creating ticket for <Mandalorian>: pizza $1': 567,
         'creating ticket for <admin acc>:

In [2]:
from expenses.models import BankAccount, Ticket

In [31]:
from django.db.models import Sum, Count

report_result_list = [['account_text', 'ticket_text', 'total_money_spent', 'total_actions']]

for acc in BankAccount.objects.all().order_by('id'):
    tickets_filtered = acc.ticket_set.all().filter(ticket_text__startswith="pizza").filter(money=10)
    
    account_text = acc.account_text
    ticket_text = tickets_filtered[0].ticket_text
    total_money_spent = list(tickets_filtered.aggregate(Sum('money')).values())[0]
    total_actions = list(tickets_filtered.aggregate(Count('money')).values())[0]
    
#     print(f"{acc.id} | {account_text} | {ticket_text} | {total_money_spent} | {total_actions}")
    
    report_result_list.append([acc.id, account_text, ticket_text, total_money_spent, total_actions])

In [32]:
report_result_list

[['account_text', 'ticket_text', 'total_money_spent', 'total_actions'],
 [1, 'admin acc', 'pizza', 7630, 763],
 [2, 'Ivan', 'pizza', 1550, 155],
 [3, 'Mandalorian', 'pizza', 3200, 320]]